# Chapter 24: Declaration Merging

---

## 24.1 Understanding Declaration Merging

Declaration merging is a unique TypeScript feature that allows the compiler to combine multiple separate declarations with the same name into a single definition. This capability bridges the gap between static type definitions and runtime values, enabling sophisticated patterns for extending types, augmenting external libraries, and organizing code.

### 24.1.1 What is Declaration Merging?

When TypeScript encounters multiple declarations with the same name in the same scope, it doesn't always treat them as duplicates or errors. Instead, for certain declaration types, it merges them into a unified entity.

```typescript
// Interface Merging - Most common form
interface User {
  name: string;
}

interface User {
  age: number;
}

// Result: User has both name and age
const user: User = {
  name: "John",
  age: 30
};

// Namespace Merging
namespace Validation {
  export function isString(value: any): value is string {
    return typeof value === "string";
  }
}

namespace Validation {
  export function isNumber(value: any): value is number {
    return typeof value === "number";
  }
}

// Both functions available on Validation
Validation.isString("hello");  // ✅
Validation.isNumber(42);     // ✅
```

**Types That Can Be Merged:**

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Declaration Merging Capabilities                  │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   Declaration Type    │ Can Merge? │ Merge Behavior                │
│  ─────────────────────┼────────────┼────────────────────────────── │
│   Interface           │ ✅ Yes     │ Members are combined          │
│   Type Alias          │ ❌ No      │ Duplicate identifier error      │
│   Class               │ ⚠️ Partial │ See namespace merging         │
│   Namespace           │ ✅ Yes     │ Exports are combined            │
│   Enum                │ ⚠️ Partial │ See namespace merging         │
│   Function            │ ⚠️ Partial │ See namespace merging         │
│   Variable (let/const)│ ❌ No      │ Duplicate declaration error     │
│                                                                     │
│   Special Cases:                                                     │
│   • Namespace + Class: Merges static members with class             │
│   • Namespace + Function: Adds properties to function object          │
│   • Namespace + Enum: Adds static members to enum                   │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

### 24.1.2 Merge Rules and Conflicts

When merging declarations, TypeScript follows specific rules for handling duplicate or conflicting members.

```typescript
// Non-conflicting merges (different properties)
interface Person {
  name: string;
}

interface Person {
  age: number;
}
// Result: { name: string; age: number; }

// Same property with same type (allowed)
interface Config {
  timeout: number;
}

interface Config {
  timeout: number;  // Same type, OK
}

// Same property with different type (ERROR)
interface Settings {
  value: string;
}

// interface Settings {
//   value: number;  // ❌ Error: Subsequent property declarations must have the same type
// }

// Function overloads through interface merging
interface Logger {
  log(message: string): void;
}

interface Logger {
  log(message: string, level: "info" | "error"): void;
}

// Result: Logger has two overloads
const logger: Logger = {
  log(message: string, level?: "info" | "error") {
    console.log(message, level);
  }
};

logger.log("Hello");           // OK - first overload
logger.log("Error", "error");  // OK - second overload
```

---

## 24.2 Merging Interfaces

Interface merging is the most straightforward and commonly used form of declaration merging. It allows you to extend interfaces across multiple declarations, which is particularly useful for augmenting third-party types or organizing large type definitions.

### 24.2.1 Interface Extension vs Merging

While `extends` creates an inheritance relationship, merging combines declarations at the same level.

```typescript
// Interface Extension (inheritance)
interface Animal {
  name: string;
}

interface Dog extends Animal {
  breed: string;
}
// Dog has: name (inherited), breed (own)

// Interface Merging (same name)
interface Cat {
  name: string;
}

interface Cat {
  color: string;
}
// Cat has: name (merged), color (merged)

// Practical difference
const dog: Dog = {
  name: "Rex",
  breed: "German Shepherd"
  // Must provide both at once
};

const cat: Cat = {
  name: "Whiskers",
  color: "orange"
  // Properties accumulated from all declarations
};

// Extension creates subtypes, merging creates single type
type DogKeys = keyof Dog;  // "name" | "breed"
type CatKeys = keyof Cat;  // "name" | "color"
```

### 24.2.2 Declaration Merging Patterns

Common patterns for using interface merging in real applications.

```typescript
// Pattern 1: Extending third-party types (module augmentation)
// In your own type definitions file:

// Original library interface (in node_modules)
// interface User {
//   id: number;
//   name: string;
// }

// Your augmentation (in your project)
interface User {
  email: string;
  role: "admin" | "user";
}

// Now User has all properties
const user: User = {
  id: 1,
  name: "John",
  email: "john@example.com",
  role: "admin"
};

// Pattern 2: Organizing large interfaces by feature
// In types.ts
interface AppConfig {
  // Core properties
  version: string;
  environment: "development" | "production";
}

// In database.ts
interface AppConfig {
  // Database section
  database: {
    host: string;
    port: number;
  };
}

// In cache.ts
interface AppConfig {
  // Cache section
  redis: {
    url: string;
    ttl: number;
  };
}

// Final AppConfig has all properties
const fullConfig: AppConfig = {
  version: "1.0.0",
  environment: "production",
  database: { host: "localhost", port: 5432 },
  redis: { url: "redis://localhost", ttl: 3600 }
};

// Pattern 3: Progressive enhancement
interface BaseResponse {
  success: boolean;
}

interface ErrorResponse {
  success: false;
  error: string;
  code: number;
}

interface SuccessResponse<T> {
  success: true;
  data: T;
}

// Merged through union, not interface merging
type ApiResponse<T> = SuccessResponse<T> | ErrorResponse;
```

### 24.2.3 Generic Interface Merging

Interfaces with the same name and type parameters merge their members, with constraints being combined.

```typescript
// Generic interface merging
interface Container<T> {
  value: T;
}

interface Container<T> {
  getValue(): T;
  setValue(newValue: T): void;
}

// Result: Container<T> has all three members
const container: Container<number> = {
  value: 42,
  getValue() { return this.value; },
  setValue(v) { this.value = v; }
};

// Constraints in merging
interface Repository<T extends { id: number }> {
  findById(id: number): T | undefined;
}

interface Repository<T extends { id: number }> {
  save(entity: T): void;
  delete(id: number): void;
}

// Both declarations must have compatible constraints
// class UserRepo implements Repository<User> { ... }
```

---

## 24.3 Merging Namespaces

Namespaces (formerly internal modules) can be merged with other namespaces, as well as with classes, functions, and enums to extend their functionality.

### 24.3.1 Namespace-to-Namespace Merging

Multiple namespace declarations with the same name merge their exported members.

```typescript
// First namespace declaration
namespace Validation {
  export function isString(value: any): value is string {
    return typeof value === "string";
  }
  
  export const StringRegex = /^[a-zA-Z]+$/;
}

// Second namespace declaration (merges with first)
namespace Validation {
  export function isNumber(value: any): value is number {
    return typeof value === "number";
  }
  
  export function isEmail(value: string): boolean {
    return value.includes("@");
  }
}

// All exports are available
Validation.isString("hello");   // ✅
Validation.isNumber(42);        // ✅
Validation.isEmail("test@example.com"); // ✅
Validation.StringRegex;         // ✅

// Non-exported members are not merged
namespace Internal {
  const secret = "hidden";  // Not exported, not visible outside
  
  export function publicFn() {
    return secret;
  }
}

// Internal.secret;  // ❌ Error: doesn't exist
Internal.publicFn();  // ✅ OK
```

### 24.3.2 Namespace Scoping and Visibility

Merged namespaces maintain separate scopes for non-exported members, while exports are combined.

```typescript
// Demonstrating scope in merged namespaces
namespace MyLib {
  // Internal helper, not exported
  function logInternal(message: string) {
    console.log(`[Internal] ${message}`);
  }
  
  export function publicFn1() {
    logInternal("Called from publicFn1");  // Can access internal
    return "result1";
  }
}

namespace MyLib {
  // Different internal helper with same name (allowed, separate scope)
  function logInternal(message: string) {
    console.log(`[Internal2] ${message}`);
  }
  
  export function publicFn2() {
    logInternal("Called from publicFn2");  // Refers to second logInternal
    return "result2";
  }
}

// Both public functions available
MyLib.publicFn1();
MyLib.publicFn2();

// MyLib.logInternal;  // ❌ Error: not exported
```

---

## 24.4 Merging Namespaces with Classes

Namespaces can merge with class declarations to add static members or organize related types without polluting the class definition.

### 24.4.1 Adding Static Members

Use namespace merging to add static properties and methods to a class.

```typescript
// Class declaration
class User {
  constructor(
    public name: string,
    public email: string
  ) {}
  
  validate(): boolean {
    return this.email.includes("@");
  }
}

// Merging namespace to add static members
namespace User {
  // Static factory method
  export function createGuest(): User {
    return new User("Guest", "guest@example.com");
  }
  
  // Static constants
  export const MAX_NAME_LENGTH = 100;
  export const EMAIL_REGEX = /^[^\s@]+@[^\s@]+\.[^\s@]+$/;
  
  // Static validation
  export function isValidEmail(email: string): boolean {
    return EMAIL_REGEX.test(email);
  }
}

// Usage
const user = new User("John", "john@example.com");
const guest = User.createGuest();           // Static method
const maxLength = User.MAX_NAME_LENGTH;     // Static property
const isValid = User.isValidEmail("test");  // Static method
```

### 24.4.2 Organizing Related Types

Keep related interfaces and types organized with the class they belong to.

```typescript
// Main class
class Observable<T> {
  private subscribers: Array<(value: T) => void> = [];
  
  subscribe(fn: (value: T) => void): void {
    this.subscribers.push(fn);
  }
  
  next(value: T): void {
    this.subscribers.forEach(fn => fn(value));
  }
}

// Namespace for related types
namespace Observable {
  // Type for subscription function
  export type Subscriber<T> = (value: T) => void;
  
  // Options interface
  export interface Options {
    bufferSize?: number;
    immediate?: boolean;
  }
  
  // Factory function
  export function create<T>(initialValue?: T): Observable<T> {
    return new Observable<T>();
  }
  
  // Utility to combine observables
  export function combine<T>(...observables: Observable<T>[]): Observable<T[]> {
    const combined = new Observable<T[]>();
    // Implementation...
    return combined;
  }
}

// Usage
const obs = Observable.create<number>();
const combined = Observable.combine(obs, obs);

// Accessing related types
type Sub = Observable.Subscriber<string>;
```

---

## 24.5 Merging Namespaces with Functions

Functions can merge with namespaces to add properties to function objects, enabling patterns like jQuery-style APIs or function factories with attached utilities.

### 24.5.1 Adding Properties to Functions

Attach static properties and methods to functions using namespace merging.

```typescript
// Function declaration
function greet(name: string): string {
  return `Hello, ${name}!`;
}

// Merging namespace to add properties
namespace greet {
  // Default greeting
  export const defaultGreeting = "Hello";
  
  // Formal greeting
  export function formal(name: string): string {
    return `Greetings, ${name}.`;
  }
  
  // Casual greeting
  export function casual(name: string): string {
    return `Hey ${name}!`;
  }
  
  // Counter for tracking
  export let count = 0;
  
  // Wrapped function with counting
  export function counted(name: string): string {
    count++;
    return greet(name);
  }
}

// Usage
console.log(greet("John"));           // "Hello, John!"
console.log(greet.formal("Jane"));    // "Greetings, Jane."
console.log(greet.casual("Bob"));     // "Hey Bob!"
console.log(greet.defaultGreeting);   // "Hello"
console.log(greet.counted("Alice"));  // "Hello, Alice!" (and increments count)
```

### 24.5.2 Function Factory Pattern

Create function factories that return callable objects with attached methods.

```typescript
// Counter factory function
function createCounter(initial = 0) {
  let count = initial;
  
  function counter() {
    return ++count;
  }
  
  counter.reset = () => {
    count = initial;
  };
  
  counter.getCount = () => count;
  
  counter.setCount = (value: number) => {
    count = value;
  };
  
  return counter;
}

// TypeScript understands the merged structure
const counter = createCounter(10);

console.log(counter());        // 11 (calling the function)
console.log(counter.getCount()); // 11
counter.setCount(100);
console.log(counter());        // 101
counter.reset();
console.log(counter());        // 11 (back to initial + 1)

// Type-safe with proper typing
type Counter = ReturnType<typeof createCounter>;
// TypeScript infers the callable signature plus attached methods
```

---

## 24.6 Merging Namespaces with Enums

Enums can merge with namespaces to add static methods and constants, extending their functionality beyond simple value collections.

### 24.6.1 Adding Static Methods to Enums

Use namespace merging to add utility methods to enum types.

```typescript
// Enum declaration
enum Status {
  Pending = "PENDING",
  Approved = "APPROVED",
  Rejected = "REJECTED"
}

// Merging namespace to add methods
namespace Status {
  // Check if status is final (not pending)
  export function isFinal(status: Status): boolean {
    return status === Status.Approved || status === Status.Rejected;
  }
  
  // Get display label
  export function getLabel(status: Status): string {
    switch (status) {
      case Status.Pending: return "⏳ Pending";
      case Status.Approved: return "✅ Approved";
      case Status.Rejected: return "❌ Rejected";
    }
  }
  
  // Parse from string
  export function fromString(value: string): Status | undefined {
    if (Object.values(Status).includes(value as Status)) {
      return value as Status;
    }
    return undefined;
  }
  
  // Get all statuses as array
  export function getAll(): Status[] {
    return [Status.Pending, Status.Approved, Status.Rejected];
  }
}

// Usage
const currentStatus = Status.Pending;

console.log(Status.isFinal(currentStatus));     // false
console.log(Status.getLabel(currentStatus));    // "⏳ Pending"
console.log(Status.fromString("APPROVED"));     // Status.Approved

// Iterating over all statuses
Status.getAll().forEach(status => {
  console.log(Status.getLabel(status));
});
```

### 24.6.2 Enum with Associated Data

Store additional metadata alongside enum values using namespace merging.

```typescript
// Enum for HTTP status codes
enum HttpStatus {
  OK = 200,
  Created = 201,
  BadRequest = 400,
  Unauthorized = 401,
  Forbidden = 403,
  NotFound = 404,
  ServerError = 500
}

// Associated data through namespace merging
namespace HttpStatus {
  interface StatusInfo {
    code: number;
    message: string;
    description: string;
    isError: boolean;
  }
  
  const statusMap: Record<HttpStatus, StatusInfo> = {
    [HttpStatus.OK]: {
      code: 200,
      message: "OK",
      description: "Request succeeded",
      isError: false
    },
    [HttpStatus.Created]: {
      code: 201,
      message: "Created",
      description: "Resource created successfully",
      isError: false
    },
    [HttpStatus.BadRequest]: {
      code: 400,
      message: "Bad Request",
      description: "Invalid request syntax",
      isError: true
    },
    [HttpStatus.Unauthorized]: {
      code: 401,
      message: "Unauthorized",
      description: "Authentication required",
      isError: true
    },
    [HttpStatus.Forbidden]: {
      code: 403,
      message: "Forbidden",
      description: "Permission denied",
      isError: true
    },
    [HttpStatus.NotFound]: {
      code: 404,
      message: "Not Found",
      description: "Resource not found",
      isError: true
    },
    [HttpStatus.ServerError]: {
      code: 500,
      message: "Internal Server Error",
      description: "Server encountered an error",
      isError: true
    }
  };
  
  export function getInfo(status: HttpStatus): StatusInfo {
    return statusMap[status];
  }
  
  export function isSuccess(status: HttpStatus): boolean {
    return !statusMap[status].isError;
  }
  
  export function getMessage(status: HttpStatus): string {
    return statusMap[status].message;
  }
}

// Usage
const status = HttpStatus.NotFound;
console.log(HttpStatus.getInfo(status));
console.log(HttpStatus.isSuccess(status));  // false
console.log(HttpStatus.getMessage(status)); // "Not Found"
```

---

## 24.7 Module Augmentation

Module augmentation allows you to extend types declared in external modules or your own modules, enabling you to add properties to existing interfaces, add new exports to modules, or fix missing type definitions.

### 24.7.1 Augmenting External Modules

Extend third-party library types to add missing functionality or custom extensions.

```typescript
// augmentations.d.ts or in your type definition file

// Original module (e.g., from node_modules/library)
// declare module "some-library" {
//   export interface Config {
//     timeout: number;
//   }
// }

// Augmenting the external module
declare module "some-library" {
  // Add properties to existing interface
  interface Config {
    retries?: number;      // New optional property
    headers?: Record<string, string>;  // New property
  }
  
  // Add new interface
  interface AdvancedConfig extends Config {
    cacheEnabled: boolean;
  }
  
  // Add new function
  export function validateConfig(config: Config): boolean;
}

// Usage in application
import { Config, validateConfig, AdvancedConfig } from "some-library";

const config: Config = {
  timeout: 5000,
  retries: 3,           // ✅ From augmentation
  headers: {}           // ✅ From augmentation
};

validateConfig(config);   // ✅ From augmentation
```

### 24.7.2 Global Augmentation

Augment global types like `Window`, `Array`, or `String` to add custom properties.

```typescript
// global.d.ts

// Augmenting the global Window object
declare global {
  interface Window {
    __APP_CONFIG__?: {
      apiUrl: string;
      version: string;
    };
    __INITIAL_STATE__?: any;
    customLib?: {
      init: () => void;
      destroy: () => void;
    };
  }
}

// Augmenting Array prototype
declare global {
  interface Array<T> {
    last(): T | undefined;
    first(): T | undefined;
    groupBy<K extends string | number | symbol>(
      keyFn: (item: T) => K
    ): Record<K, T[]>;
  }
}

// Implementing the extensions
if (!Array.prototype.last) {
  Array.prototype.last = function<T>(this: T[]): T | undefined {
    return this[this.length - 1];
  };
}

if (!Array.prototype.first) {
  Array.prototype.first = function<T>(this: T[]): T | undefined {
    return this[0];
  };
}

// Usage
const arr = [1, 2, 3];
const last = arr.last();    // ✅ TypeScript knows it's number | undefined
const first = arr.first();  // ✅ TypeScript knows it's number | undefined

// Accessing augmented window
if (window.__APP_CONFIG__) {
  console.log(window.__APP_CONFIG__.apiUrl);
}

export {};  // Make this a module to allow global augmentation
```

### 24.7.3 Augmentation Best Practices

Follow these guidelines to ensure safe and maintainable module augmentation.

```typescript
// ✅ DO: Use declaration merging for extending third-party types
// when the library doesn't expose certain properties you need

// ✅ DO: Place augmentations in .d.ts files or dedicated files
// like types/augmentations.d.ts

// ✅ DO: Export empty object to make file a module if needed
export {};

// ✅ DO: Use specific module names in declare module
declare module "express-serve-static-core" {
  interface Request {
    user?: User;  // Add user property to Express Request
  }
}

// ❌ DON'T: Augment modules you don't own without good reason
// This can break with library updates

// ❌ DON'T: Use augmentation to fix incorrect types in libraries
// Instead, contribute fixes upstream or use type assertions

// ❌ DON'T: Augment global objects excessively
// This pollutes the global namespace and can cause conflicts

// ✅ DO: Document why augmentation is needed
/**
 * Augments Express Request to include user property
 * added by authentication middleware
 */
declare module "express" {
  interface Request {
    user?: {
      id: string;
      email: string;
    };
  }
}

// ✅ DO: Use module augmentation for plugin architectures
// Plugin adds methods to core class
class Core {
  doCoreThing() {}
}

namespace Core {
  export interface Plugin {
    install(core: Core): void;
  }
}

// Plugin augments Core
namespace Core {
  export function use(plugin: Plugin) {
    plugin.install(new Core());
  }
}
```

---

## 24.8 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored declaration merging, TypeScript's unique ability to combine multiple declarations:

**Key Takeaways:**

1. **Declaration Merging Basics**:
   - Allows combining multiple declarations with the same name
   - Works with interfaces, namespaces, classes, functions, and enums
   - Does not work with type aliases or variable declarations

2. **Interface Merging**:
   - Most common form of declaration merging
   - Members are combined into a single interface
   - Non-function members must be unique in type
   - Function members are overloaded

3. **Namespace Merging**:
   - Exported members from all declarations are combined
   - Non-exported members remain scoped to their original declaration
   - Useful for organizing code across multiple files

4. **Class/Function/Enum + Namespace Merging**:
   - Adds static-like properties to classes and functions
   - Extends enums with utility functions
   - Creates callable objects with properties (e.g., jQuery-style APIs)

5. **Module Augmentation**:
   - Extend third-party library types
   - Add properties to global objects (Window, Array, etc.)
   - Fix missing or incorrect type definitions
   - Enable plugin architectures

### Practical Exercises

**Exercise 1: Interface Merging**

Practice merging interfaces:

```typescript
// 1. Create two interface declarations for 'Product' that merge to create:
//    - First declaration: id, name, price
//    - Second declaration: description, inStock
//    - Verify that a Product object can have all properties

// 2. Create a 'Logger' interface with two overloads through merging:
//    - First: log(message: string): void
//    - Second: log(message: string, level: "info" | "error"): void
//    - Test that both signatures work

// 3. Attempt to merge interfaces with conflicting property types:
//    - First: interface Config { timeout: number; }
//    - Second: interface Config { timeout: string; }
//    - Observe the error and explain why it occurs
```

**Exercise 2: Namespace Merging**

Implement namespace merging patterns:

```typescript
// 1. Create a 'MathUtils' namespace across two declarations:
//    - First: export function add(a: number, b: number): number
//    - Second: export function multiply(a: number, b: number): number
//    - Verify both functions are available

// 2. Create a namespace with non-exported members:
//    - First namespace: const SECRET = "hidden"; export function reveal(): string
//    - Second namespace: const SECRET = "also hidden"; export function hide(): string
//    - Demonstrate that each function accesses its own scope's SECRET

// 3. Create a class 'Widget' that merges with a namespace:
//    - Class: constructor, instance methods
//    - Namespace: static create() factory method, static VERSION property
//    - Demonstrate usage of both instance and static members
```

**Exercise 3: Module Augmentation**

Practice extending external types:

```typescript
// 1. Create a module augmentation for a fictional 'express' module:
//    - Add 'user' property to Request interface (with id, email)
//    - Add 'requestTime' property to Request (Date)
//    - Write a middleware function that populates these properties

// 2. Augment the global Array interface:
//    - Add 'sum()' method for number arrays
//    - Add 'average()' method for number arrays
//    - Provide implementations using declaration merging

// 3. Create augmentation for 'vue' module (if familiar) or create a hypothetical module:
//    - Add custom directive types
//    - Extend component options
//    - Ensure type safety in component definitions
```

**Exercise 4: Real-World Application**

Build a type-safe plugin system using declaration merging:

```typescript
// Create a 'PluginSystem' architecture:

// 1. Core class 'Application' with basic methods
//    - constructor, start(), stop()
//    - Merge with namespace to add static plugin management

// 2. Plugin interface definition
//    - name: string
//    - install(app: Application): void
//    - Optional: configure(options: any): void

// 3. Module augmentation for plugins
//    - Allow plugins to augment Application with new methods
//    - Example: DatabasePlugin adds db property, AuthPlugin adds auth property

// 4. Type-safe plugin usage
//    - app.use(DatabasePlugin) should make app.db available
//    - app.use(AuthPlugin) should make app.auth available
//    - TypeScript should track which plugins are installed

// Implementation should demonstrate:
// - Class + namespace merging
// - Interface merging for plugin contracts
// - Module augmentation for extending Application
// - Generic constraints for type-safe plugin installation
```

### Additional Resources

- **TypeScript Handbook - Declaration Merging**: https://www.typescriptlang.org/docs/handbook/declaration-merging.html
- **TypeScript Module Augmentation**: https://www.typescriptlang.org/docs/handbook/declaration-merging.html#module-augmentation
- **DefinitelyTyped Contribution Guide**: https://definitelytyped.org/guides/contributing.html (for module augmentation examples)
- **TypeScript Deep Dive - Declaration Merging**: https://basarat.gitbook.io/typescript/type-system/declaration-merging

---

## Coming Up Next: Chapter 25 - Modules

In the next chapter, we will explore **Modules**, TypeScript's system for organizing code into reusable, encapsulated units:

- **25.1 Understanding Modules** - Module vs. script mode, module resolution
- **25.2 Exporting** - Named exports, default exports, re-exports
- **25.3 Importing** - Named imports, default imports, namespace imports
- **25.4 Module Resolution Strategies** - Classic vs. Node.js resolution
- **25.5 Path Mappings and Aliases** - Configuring module paths
- **25.6 ES Modules vs CommonJS** - Interoperability and differences

Modules are the foundation of modern TypeScript applications, enabling code organization, dependency management, and scalable architecture. Understanding module systems is essential for building maintainable applications and libraries.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='23. type_inference.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../7. modules_and_namespaces/25. modules.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
